# Load ImageNet images
Imagenet images are subject to copyright and some could image urls may be outdated and invalid. So there's a chance that the image could not be obtained. 

In [4]:
import numpy as np
import requests
import cv2
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlretrieve
import json
import os.path

bold_dir = "/home/johnson/Documents/BOLD5000"

In [2]:
def geturl(imageName):
    wnid = imageName[:imageName.find('_')]
    image_urls = "http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=" + wnid
    url_mappings = "http://www.image-net.org/api/text/imagenet.synset.geturls.getmapping?wnid=" + wnid
    '''
    page = requests.get(image_urls)
    soup = BeautifulSoup(page.content, 'html.parser')
    str_soup = str(soup)
    urls = str_soup.split('\r\n')
    #print(urls[:100])
    '''
    imgName = imageName[:imageName.find('.')]
    try:
        mapping_page = requests.get(url_mappings)
    except Exception as e:
        print(e)
    mapping_soup = BeautifulSoup(mapping_page.content, 'html.parser')
    mapping_str = str(mapping_soup)
    
    i = mapping_str.find(imgName)
    if i == -1:
        return False
    start_index = mapping_str.find(' ', i) + 1
    end_index = mapping_str.find('\r\n', start_index)
    url = mapping_str[start_index:end_index]
    
    # Parse unicode string to python-safe url
    url = urllib.parse.quote(url, safe=':/')
    return url
    
#     mappings = mapping_str.split('\r\n')
#     for mapping in mappings:
#         name = mapping[:mapping.find(' ')]
#         #print(imageName[:imageName.find('.')])
#         if name == imgName:
#             url = mapping[mapping.find(' ')+1:]
#             return url
#     return False

In [3]:
geturl('n07695742_5848.JPEG')

'http://farm4.static.flickr.com/3561/3330359073_e07d7c4f90.jpg'

## 1×× Informational
100 Continue
101 Switching Protocols
102 Processing
## 2×× Success
200 OK
201 Created
202 Accepted
203 Non-authoritative Information
204 No Content
205 Reset Content
206 Partial Content
207 Multi-Status
208 Already Reported
226 IM Used
## 3×× Redirection
300 Multiple Choices
301 Moved Permanently
302 Found
303 See Other
304 Not Modified
305 Use Proxy
307 Temporary Redirect
308 Permanent Redirect
## 4×× Client Error
400 Bad Request
401 Unauthorized
402 Payment Required
403 Forbidden
404 Not Found
405 Method Not Allowed
406 Not Acceptable
407 Proxy Authentication Required
408 Request Timeout
409 Conflict
410 Gone
411 Length Required
412 Precondition Failed
413 Payload Too Large
414 Request-URI Too Long
415 Unsupported Media Type
416 Requested Range Not Satisfiable
417 Expectation Failed
418 I'm a teapot
421 Misdirected Request
422 Unprocessable Entity
423 Locked
424 Failed Dependency
426 Upgrade Required
428 Precondition Required
429 Too Many Requests
431 Request Header Fields Too Large
444 Connection Closed Without Response
451 Unavailable For Legal Reasons
499 Client Closed Request
## 5×× Server Error
500 Internal Server Error
501 Not Implemented
502 Bad Gateway
503 Service Unavailable
504 Gateway Timeout
505 HTTP Version Not Supported
506 Variant Also Negotiates
507 Insufficient Storage
508 Loop Detected
510 Not Extended
511 Network Authentication Required
599 Network Connect Timeout Error

In [11]:

            
for session in range(0,4):
    # Open the stimlist for reading
    with open('{}/BOLD5000_ROIs/ROIs/stim_lists/CSI0{}_stim_lists.txt'.format(bold_dir, session+1), 'r') as f:
        stimlist = f.read()
    img_folder_url = "{}/BOLD5000_ROIs/ROIs/CSI{}/images".format(bold_dir, session+1)
    img_list = stimlist.split('\n')
    if not os.path.isdir(img_folder_url):
        os.mkdir(img_folder_url)
    #data = {}
    for img_name in img_list:
        # For some reason the last string is empty and incurs error
        img_dir = os.path.join(img_folder_url, img_name)
        if (img_name[0] == 'n') and (len(img_name)>0):
            if os.path.isfile(img_dir):
                print('{} already exists!'.format(img_name))
                continue
            if img_name in data:
                print('Already in dict')
                continue
            img_url = geturl(img_name)
            # Could not find image url in mapping
            if img_url == False:
                print("Img url invalid")
                data[img_name] = ""
                continue
            # Get status code 
            try:
                header = requests.head(img_url, allow_redirects=True)
                code = header.status_code
            except requests.ConnectionError:
                print("failed to connect")
                data[img_name] = ""
            except Exception as e:
                print("Other errors {}".format(e))

            if (int(code/100)==2) or (int(code/100)==3) or (int(code%100)==1):
                data[img_name] = img_url
                try:
                    urlretrieve(img_url,img_dir)
                except Exception as e:
                    print('------------------')
                    print(e)
                    print(img_url)
                    print('Status code {}'.format(code))
                    data[img_name] = ""
            else:
                print('------------------')
                print(img_url)
                print('Status code {}'.format(code))
                data[img_name] = ""
                    
    with open('{}/BOLD5000_ROIs/ROIs/CSI{}.json'.format(bold_dir, session+1), 'w') as f:
        json.dump(data, f)

Already in dict
Already in dict
n07695742_5848.JPEG already exists!
Already in dict
Already in dict
Already in dict
Already in dict
Already in dict
n04371430_12487.JPEG already exists!
Already in dict
n02655020_1972.JPEG already exists!
Already in dict
Already in dict
Already in dict
n02948072_4353.JPEG already exists!
n02219486_30442.JPEG already exists!
n03249569_23070.JPEG already exists!
n02096585_8512.JPEG already exists!
n03857828_46491.JPEG already exists!
n02510455_1670.JPEG already exists!
n03594945_19394.JPEG already exists!
Already in dict
Already in dict
n02457408_8048.JPEG already exists!
Already in dict
Already in dict
Already in dict
n01689811_365.JPEG already exists!
n01531178_2651.JPEG already exists!
Already in dict
Already in dict
n03791053_3596.JPEG already exists!
Already in dict
n04404412_6.JPEG already exists!
n04330267_289.JPEG already exists!
n03843555_6225.JPEG already exists!
n07745940_20689.JPEG already exists!
Already in dict
n03599486_2956.JPEG already exi

Other errors Exceeded 30 redirects.
------------------
http://www.dooziedog.com/dog_breeds/images/full/Golden-Retriever-2.jpg
Status code 403
failed to connect
------------------
hostname 'thumb10.shutterstock.com.edgesuite.net' doesn't match either of 'a248.e.akamai.net', '*.akamaized-staging.net', '*.akamaized.net', '*.akamaihd-staging.net', '*.akamaihd.net'
http://thumb10.shutterstock.com.edgesuite.net/display_pic_with_logo/180238/180238%2C1206740297%2C1/stock-photo-close-up-of-surface-fur-leopard-panthera-pardus-10882735.jpg
Status code 200
------------------
HTTP Error 403: Forbidden
http://www.commercial-furniture.com/products/wardrobe.jpg
Status code 200
------------------
http://farm4.static.flickr.com/3189/2716303657_1a3b4dc9f4.jpg
Status code 404
------------------
http://www.wbcnet.de/tips/bilder/screwdriver-combi.jpg
Status code 404
failed to connect
------------------
<urlopen error [Errno -2] Name or service not known>
http://media.dealerfusion.com/images/001329-12433B.jp

------------------
http://www.herbarium.iastate.edu/fungi/images/Ramaria%2520botrytis%25204%2520Mann%2520Aug%25202004.jpg
Status code 404
------------------
http://static.flickr.com/107/257651535_a5ae85f0ff.jpg
Status code 404
failed to connect
------------------
http://www.cdnews.gov.cn/image20010518/8610.jpg
Status code 404
Img url invalid
------------------
http://farm1.static.flickr.com/180/427335672_71c1ae63c9.jpg
Status code 410
failed to connect
------------------
http://www.mackillinois.com/img/trucks/183_lg.jpg
Status code 410
------------------
http://cn.made-in-china.com/image/2f0j01cesaplqEIQgzM/%E7%B2%BE%E7%BE%8E%E5%A9%9A%E7%BA%B1%E7%A4%BC%E6%9C%8D%2B%EF%BC%88OEM-W231%EF%BC%89.jpg
Status code 405
Img url invalid
------------------
http://farm1.static.flickr.com/99/265440857_a6310c7aa0.jpg
Status code 410
------------------
http://www.awesomegsps.com/shasta.jpg
Status code 403
failed to connect
------------------
<urlopen error [Errno -2] Name or service not known>
http://w

failed to connect
------------------
http://ribeyejaxs.com/images/SteakBurrito.jpg
Status code 404
------------------
http://vanderhaagen.eu/nieuws_files/aug2007%252520011.jpg
Status code 404
------------------
http://www.michiganlights.com/LSS%2520Photos/Grand%2520Haven%2520CG%2520Sta%2520Boathouse%25202006.jpg
Status code 404
Img url invalid
failed to connect
------------------
<urlopen error [Errno -2] Name or service not known>
http://dl.dogomania.com/pics/28/Cocoa_2.jpg
Status code 200
------------------
http://farm3.static.flickr.com/2384/1974319639_8906e18730.jpg
Status code 410
------------------
http://www.annonces.webannuaire.org/image_annonce/85_15-12-2006_macaque%2520dassa.JPG
Status code 404
------------------
http://www.nce.co.uk/pictures/586xAny/9/9/8/1213998_jackup_credit_John_Fawcett.jpg
Status code 403
------------------
http://thesinksdiscgolf.com/images/TRASH.JPG
Status code 404
------------------
http://www.bluedogengineering.com/Bluedog-Tractors/forklifts/KOMATSU-

------------------
http://www.freqofnature.com/photos/lafd_fireboat2/pa202778.jpg
Status code 404
------------------
http://farm3.static.flickr.com/2007/2054859692_d9ffd2cdd7.jpg
Status code 404
------------------
http://i.pbase.com/u12/bernd001/upload/21764061.MyDocumentsGraphicArtisticForReviewTempRV.jpg
Status code 403
failed to connect
------------------
http://www.jensenscabinetshop.com/index.php/main.content.image/sawmill01_01.jpg
Status code 403
------------------
http://farm1.static.flickr.com/48/113465698_406d17d9dc.jpg
Status code 410
failed to connect
------------------
<urlopen error [Errno 110] Connection timed out>
http://images.onlinegalleries.com:8080/gfx/42228.jpg
Status code 200
------------------
http://i.pbase.com/o6/79/695179/1/75053491.anTaQFdI.koribustardbird_MG_9306.jpg
Status code 403
failed to connect
------------------
http://www.fittingandturning.co.nz/parrallel%2520004.jpg
Status code 403
------------------
http://www.kzst.com/clubz/prizes/February/feb_whal

------------------
http://farm3.static.flickr.com/2406/2034132386_70a375da04.jpg
Status code 410


IndexError: string index out of range

In [115]:
with open('/Users/jz/Desktop/BOLD5000/BOLD5000_ROIs/ROIs/CSI{}.json'.format(0+1), 'w') as f:
    json.dump(data, f)

In [122]:
fails = 0
successes = 0
for (key, value) in data.items():
    if value == '':
        fails += 1
    else:
        successes += 1
print(successes, "successes")
print(fails, "fails")

1060 successes
635 fails


In [134]:
len(img_list)

5255

In [ ]:
img_list[:100]